In [1]:
from google.colab import drive

drive.mount('/content/drive')

!unzip -q "/content/drive/MyDrive/extracted.zip" -d "/content/drive/MyDrive/osic-pulmonary-fibrosis-progression"


Mounted at /content/drive


# CNN - extract features

Pre-process image

In [2]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pydicom
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import seaborn as sns
import math
import cv2
import pydicom
import os
from tqdm import tqdm
import glob
import pickle as pkl
import matplotlib.image as mpimg
from tabulate import tabulate
import missingno as msno
from IPython.display import display_html
from PIL import Image
import gc
from skimage.transform import resize
import copy
import re
from scipy.stats import pearsonr

# Segmentation
import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

#------------------------------------------------------------

pd.set_option("display.max_columns", 100)

#Define resolution of image 512*512
N_ROWS = 512
N_COLS = 512
path = 'C:/Users/frank/OneDrive/Desktop/Thesis - Progress Prediction/CodeDevelopment/osic-pulmonary-fibrosis-progression/'
#Define Batch size
BATCH_SIZE=128

# Areas with the same number of pixels on the edges are not required. Crop it.
def crop_image(img: np.ndarray):
    edge_pixel_value = img[0, 0]
    mask = img != edge_pixel_value
    return img[np.ix_(mask.any(1),mask.any(0))]

# Load images, crop thick borders(if any) and resize
def load_image(path):
    dataset = pydicom.dcmread(path)
    img = dataset.pixel_array
    img = crop_image(img)
#   img = cv2.resize(img, (512,512))
    return img

# Get Nth percentile image
def get_img(perc, patient_id, data):

    l = glob.glob(path+'{0}/{1}/*.dcm'.format(data, patient_id))
    img_ids = []
    for x in l:
        y = x.split('\\')[-1]
        z = int(y.split('.')[0])
        img_ids.append(z)

    img_ids.sort()

    return img_ids[math.ceil(perc*(len(img_ids)))-1]

# Get num of slices bw two percentiles
def num_img_bw_perc(p1, p2, patient_id, data):

    l = glob.glob(path+'{0}/{1}/*.dcm'.format(data, patient_id))
    img_ids = []
    for x in l:
        y = x.split('\\')[-1]
        z = int(y.split('.')[0])
        img_ids.append(z)

    img_ids.sort()

    return len(img_ids[math.ceil(p1*(len(img_ids)))-1:math.ceil(p2*(len(img_ids)))])-1


# Get number of images per patient
def get_num_images(patient_id, data):

    return len(glob.glob(path+'{0}/{1}/*.dcm'.format(data, patient_id)))

# Get the lung area in the image slice
def lung_seg_pixel_ratio(img_array):

    c = 0
    for i in range(img_array.shape[0]):
        for j in range(img_array.shape[1]):
            if img_array[i][j] != 0:
                c+=1

    return c, round(c/(img_array.shape[0]*img_array.shape[1]),4)

# Get dicom meta data
def get_dicom_meta(path):

    '''Get information from the .dcm files.
    path: complete path to the .dcm file'''

    image_data = pydicom.dcmread(path)

    # Dictionary to store the information from the image
    observation_data = {
        "SliceThickness" : float(image_data.SliceThickness),
        "PixelSpacing" : float(image_data.PixelSpacing[0]),
    }

    return observation_data

# Get tissue mask
# To extract the tissues from the segmented lung all we need to do is get rid of the border parts from the segmented lung
# Grey pixels present within the border of the lung is assumed to be tissue.
# Inorder to get rid of the border pixels of the lung we slightly perturb the segmented lung to the right, left, top and bottom
# The intersection of all the perturbed images gets rid of the border lung pixels
# This resultant image serves as the mask for the tissue segmentation
def tissue_mask(img, mask, shift_perc):

    r_dim, c_dim = img.shape[0], img.shape[1]

    # Move the image by shift_perc to the left
    del_left_cols = int(shift_perc*c_dim)

    mask1, mask2 = mask.copy(), np.zeros((r_dim, c_dim)).astype(int)
    mask1 = mask1[:,del_left_cols:]
    mask2[:,:c_dim-del_left_cols] = mask1

    # Move the image by shift_perc to the right
    del_right_cols = int(shift_perc*c_dim)

    mask3, mask4 = mask.copy(), np.zeros((r_dim, c_dim)).astype(int)
    mask3 = mask3[:,:c_dim-del_right_cols]
    mask4[:,del_right_cols:] = mask3

    # Move the image by shift_perc to the top
    del_top_rows = int(shift_perc*c_dim) #BUG C_DIM SHOULD BE R_DIM

    mask5, mask6 = mask.copy(), np.zeros((r_dim, c_dim)).astype(int)
    mask5 = mask5[del_top_rows:,:]
    mask6[:r_dim-del_top_rows,:] = mask5

    # Move the image by shift_perc to the bottom
    del_bottom_rows = int(shift_perc*r_dim)

    mask7, mask8 = mask.copy(), np.zeros((r_dim, c_dim)).astype(int)
    mask7 = mask7[:r_dim-del_bottom_rows,:]
    mask8[del_bottom_rows:,:] = mask7

    #Obtain the final mask
    final_mask = ((mask2==1) & (mask4==1) & (mask6==1) & (mask8==1)).astype(int)

    return final_mask

# Get tissue features
def tissue_features(tissue_mask, img, thresh = 0.35):

    final_img = tissue_mask*img

    checker = np.zeros((final_img.shape[0], final_img.shape[1]))
    counter, other_counter = 0, 0
    for i in range(final_img.shape[0]):
        for j in range(final_img.shape[1]):
            if final_img[i][j]>=thresh:
                checker[i][j] = 1
                counter+=1
            else:
                checker[i][j] = 0
                other_counter+=1

    tissue_by_total = counter/(final_img.shape[0]**2)
    tissue_by_lung = counter/((tissue_mask==1).sum())

    return counter, tissue_by_total, tissue_by_lung, checker

# https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/
def make_lungmask(img, display=False):

    img = img.astype(float)
    row_size= img.shape[0]
    col_size = img.shape[1]


    #Normalize image pixels
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std

    # Find the average pixel value near the lungs
    # to renormalize washed out images
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)]
    mean = np.mean(middle)
    max = np.max(img)
    min = np.min(img)

    # To improve threshold finding, I'm moving the
    # underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    #Substitutes too light/dark pixels with mean
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate to include some of the pixels surrounding the lung.
    # We don't want to accidentally clip the lung.

    eroded = morphology.erosion(thresh_img,np.ones([3,3]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))

    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if ((B[2]-B[0]<row_size*0.9) and (B[3]-B[1]<col_size*0.9) and (B[2]-B[0]>row_size*0.20)
            and (B[3]-B[1]>col_size*0.10) and (B[0]>row_size*0.03) and (B[2]<row_size*0.97)
            and (B[1]>col_size*0.03) and (B[3]<col_size*0.97)):
            good_labels.append(prop.label)
    mask = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0

    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask

    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation
 # Compute Lung Area in the slice
    lung_pixels, slice_lung_area = lung_seg_pixel_ratio(mask)

    # Tissue Mask
    t_mask = tissue_mask(img, mask, shift_perc = 0.02)

    # Extract tissue features
    num_t_pixels, tissue_by_total, tissue_by_lung, checker = tissue_features(t_mask, img, thresh = 0.35)

    if (display):
        fig, ax = plt.subplots(4, 2, figsize=[18, 18])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        ax[2, 1].axis('off')
        ax[3, 0].set_title("Inner Lung Mask")
        ax[3, 0].imshow(t_mask, cmap='gray')
        ax[3, 0].axis('off')
        ax[3, 1].set_title("Segmented Tissue")
        ax[3, 1].imshow(checker, cmap='gray')
        ax[3, 1].axis('off')

        plt.show()

    return img,thresh_img,dilation,labels,mask,mask*img,t_mask,checker




Training

In [5]:
csv_path = '/content/drive/MyDrive/patient_event_slice.csv'
df = pd.read_csv(csv_path)
print(df.head())

                     Patient  event  time  \
0  ID00007637202177411956430      1    11   
1  ID00009637202177434476278      1    52   
2  ID00010637202177584971671      1     5   
3  ID00011637202177653955184      0    52   
4  ID00012637202177665765362      1    54   

                                         slice_files  
0  ['18.dcm', '19.dcm', '2.dcm', '20.dcm', '21.dc...  
1  ['205.dcm', '216.dcm', '226.dcm', '237.dcm', '...  
2  ['32.dcm', '35.dcm', '38.dcm', '40.dcm', '43.d...  
3  ['18.dcm', '19.dcm', '2.dcm', '20.dcm', '21.dc...  
4  ['22.dcm', '24.dcm', '25.dcm', '27.dcm', '28.d...  


In [6]:
import ast
df['slice_files'] = df['slice_files'].apply(ast.literal_eval)


In [7]:
from sklearn.model_selection import train_test_split

patients = df['Patient'].tolist()

# Split 70/15/15 stratified by event
train_patients, temp_patients = train_test_split(
    patients, test_size=0.3, random_state=42, stratify=df['event']
)

val_patients, test_patients = train_test_split(
    temp_patients, test_size=0.5, random_state=42,
    stratify=df.set_index('Patient').loc[temp_patients]['event']
)

train_df = df[df['Patient'].isin(train_patients)].reset_index(drop=True)
val_df   = df[df['Patient'].isin(val_patients)].reset_index(drop=True)
test_df  = df[df['Patient'].isin(test_patients)].reset_index(drop=True)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


Train: 123, Val: 26, Test: 27


In [8]:
print(train_df)

                       Patient  event  time  \
0    ID00009637202177434476278      1    52   
1    ID00010637202177584971671      1     5   
2    ID00011637202177653955184      0    52   
3    ID00012637202177665765362      1    54   
4    ID00015637202177877247924      1     9   
..                         ...    ...   ...   
118  ID00419637202311204720264      1    12   
119  ID00421637202311550012437      0    55   
120  ID00422637202311677017371      1    41   
121  ID00423637202312137826377      1     1   
122  ID00426637202313170790466      0    59   

                                           slice_files  
0    [205.dcm, 216.dcm, 226.dcm, 237.dcm, 248.dcm, ...  
1    [32.dcm, 35.dcm, 38.dcm, 40.dcm, 43.dcm, 46.dc...  
2    [18.dcm, 19.dcm, 2.dcm, 20.dcm, 21.dcm, 22.dcm...  
3    [22.dcm, 24.dcm, 25.dcm, 27.dcm, 28.dcm, 3.dcm...  
4    [259.dcm, 275.dcm, 291.dcm, 307.dcm, 321.dcm, ...  
..                                                 ...  
118  [17.dcm, 18.dcm, 19.dcm, 2.dcm,

Dataset

In [9]:
root_path = '/content/drive/MyDrive/osic-pulmonary-fibrosis-progression/extracted'


In [ ]:
#Passare alla CNN le immagini elaborate da preprocess per aver immagini piu chiare oltre alle maschere in modo che focalizza sui polmoni


In [10]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 6.6 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=3e566c8743580a8d4295c994dd217e4c88977ea4a8a677b2eac854891924f15e
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pydicom
import numpy as np
import cv2
import os
from tqdm import tqdm
import pandas as pd
import timm
from sklearn.metrics import roc_auc_score, accuracy_score
from lifelines.utils import concordance_index  # pip install lifelines

Dataset Class


In [ ]:
!pip install pylibjpeg pylibjpeg-libjpeg python-gdcm


In [19]:
class LungEventDataset(Dataset):

  def __init__(self, df, root_path, transform=None, use_masks=True):
        self.df = df.reset_index(drop=True)
        self.root_path = root_path
        self.transform = transform
        self.use_masks = use_masks

  def __len__(self):
        return len(self.df)

  def segment_lung(self,img):
      img,thresh_img,dilation,labels,mask,mask_img,t_mask,checker = make_lungmask(img)
      return mask

  def preprocess_slice(self, img):
    img,_,_,_,_,_,_,_ = make_lungmask(img)

    # Resize to fixed size
    img_resized = cv2.resize(img, (224, 224))

    # Convert to 3 channels
    img_rgb = np.stack([img_resized]*3, axis=0)

    # ImageNet normalization
    mean = np.array([0.485, 0.456, 0.406])[:, None, None]
    std = np.array([0.229, 0.224, 0.225])[:, None, None]
    img_rgb = (img_rgb - mean) / std

    return img_rgb

  def __getitem__(self, index):
      row = self.df.iloc[index]
      patient = row['Patient']
      slice_files = row['slice_files']

      #Targets
      event = torch.tensor(row['event'], dtype= torch.float32)
      time = torch.tensor(row['time'], dtype = torch.float32)

      #Process slices
      slices = []
      for f in slice_files:
        path = os.path.join(self.root_path,patient,f)


        try:
          # Read DICOM
          ds = pydicom.dcmread(path)
          img = ds.pixel_array.astype(np.float32)

          img_rgb = self.preprocess_slice(img)
          slices.append(img_rgb)
        except Exception as e:
          print(f'Error:{e}')
          continue

      if len(slices) == 0:
        print(f'\nNo slices for {patient}')


      NUM_SLICES = 11

      if len(slices) == 0:
          # fallback to zero tensor
          slices = np.zeros((NUM_SLICES, 3, 224, 224), dtype=np.float32)
      else:
          slices = np.stack(slices)
          if slices.shape[0] < NUM_SLICES:
              pad_shape = (NUM_SLICES - slices.shape[0], *slices.shape[1:])
              slices = np.concatenate([slices, np.zeros(pad_shape, dtype=slices.dtype)], axis=0)
          elif slices.shape[0] > NUM_SLICES:
              slices = slices[:NUM_SLICES]

      slices = torch.from_numpy(slices).float()

      return{
          'slices':slices,
          'event':event,
          'time':time,
          'patient':patient,
      }




CNN

In [13]:
class SurvivalModel(nn.Module):
  #Add here clinical_Dim to insert handcrafted features
  def __init__(self, backbone_name='efficientnet_b0',pretrained = True, use_attention = True):
    super().__init__()

    #Imaging branch
    self.backbone = timm.create_model(backbone_name,pretrained=pretrained,in_chans=3,num_classes=0)

    self.feature_dim = self.backbone.num_features
    #Attention to weight the importance of slices
    self.use_attention = use_attention
    if use_attention:
      self.slice_attention = nn.Sequential(nn.Linear(self.feature_dim,128), nn.Tanh(),nn.Linear(128,1))

    #Clinical branch for later

    #Output head
    self.risk_head = nn.Linear(self.feature_dim,1) #Cox risk

  def forward(self, slices):
    B,N,C,H,W = slices.shape
    slices_flat = slices.view(B*N,C,H,W)

    # Extract features
    img_features = self.backbone(slices_flat)  # [B*N, feature_dim]
    img_features = img_features.view(B,N,self.feature_dim)  # [B,N,feature_dim]

    if self.use_attention:
        attention_logits = self.slice_attention(img_features)
        attention_weights = F.softmax(attention_logits,dim=1)
        img_pooled = (img_features * attention_weights).sum(dim=1)
    else:
        img_pooled = img_features.mean(dim=1)  # [B, feature_dim]

    # Optional reduction
    # img_pooled = F.relu(self.fc_reduce(img_pooled))  # if using intermediate layer

    risk_score = self.risk_head(img_pooled)  # [B,1]

    return risk_score



COX LIKELIHOOD LOSS

In [14]:
def cox_loss(risk_scores,times,events):
  sorted_indices = torch.argsort(times,descending=True)
  risk_scores = risk_scores[sorted_indices]
  events = events[sorted_indices]

  hazard_ratio = torch.exp(risk_scores)
  log_risk = torch.log(torch.cumsum(hazard_ratio,dim=0)+ 1e-8)

  uncensored_likelihood = risk_scores - log_risk
  loss = -torch.sum(uncensored_likelihood*events)/ (torch.sum(events)+ 1e-8)

  return loss



TRAIN

In [15]:
def train_epoch(model, loader, optimizer, device, scaler=None):
    model.train()
    total_loss = 0

    if scaler is None:
        scaler = torch.cuda.amp.GradScaler()  # initialize if not passed

    for batch in tqdm(loader, desc='Training'):
        slices = batch['slices'].to(device)
        events = batch['event'].to(device)
        times= batch['time'].to(device)

        optimizer.zero_grad()

        # Mixed precision forward
        with torch.cuda.amp.autocast():
            risk_scores = model(slices).squeeze(1)
            loss = cox_loss(risk_scores,times, events)

        # Backward with scaling
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    return total_loss / len(loader), scaler


In [16]:
def evaluate(model, loader, device):
    model.eval()
    all_risks, all_times,all_events = [], [], []

    for batch in tqdm(loader, desc='Validation'):
        slices = batch['slices'].to(device)
        events = batch['event'].to(device)
        times = batch['time'].to(device)

        with torch.cuda.amp.autocast():
            risk_scores = model(slices).squeeze(1)

        all_risks.append(risk_scores.cpu())
        all_events.append(events.cpu())
        all_times.append(times.cpu())

    all_risks = torch.cat(all_risks)
    all_events = torch.cat(all_events)

    c_index = concordance_index(all_times.numpy(), -all_risks.numpy(),all_events.numpy())
    return c_index, all_risks,all_times, all_events

MAIN

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#Load data -> already have train_df,val_df,test_df
train_dataset = LungEventDataset(train_df,root_path=root_path,use_masks=True)
val_dataset = LungEventDataset(val_df, root_path=root_path,use_masks=True)

#DataLoader
train_loader = DataLoader(train_dataset, batch_size=8,shuffle=True,num_workers=4)
val_loader = DataLoader(val_dataset,batch_size=8,shuffle=False,num_workers=4)

#Model

model = SurvivalModel( backbone_name='efficientnet_b0',pretrained=True,use_attention=True).to(device)


print(f"✅ Model initialized")
print(f"   Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3)

scaler = torch.cuda.amp.GradScaler()
best_c_index = 0.0
num_epochs = 30

for epoch in range(num_epochs):
    print(f"\n{'='*60}")
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"{'='*60}")

    # Train
    train_loss, scaler = train_epoch(model, train_loader, optimizer, device, scaler)

    # Validate
    val_c_index, val_risks, val_times, val_events = evaluate(model, val_loader, device)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val C-index: {val_c_index:.4f}")

    # Scheduler step
    scheduler.step(val_c_index)

    # Save best model
    if val_c_index > best_c_index:
        best_c_index = val_c_index
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'c_index': val_c_index,
        }, 'best_survival_model.pth')
        print(f"✅ Saved best model (C-index: {val_c_index:.4f})")

print(f"\n🎉 Training completed!")
print(f"   Best C-index: {best_c_index:.4f}")



cuda
✅ Model initialized
   Parameters: 4,172,926

Epoch 1/30


Training:  75%|███████▌  | 12/16 [10:21<02:03, 30.91s/it]

Error:Unable to decompress 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])' pixel data because all plugins are missing dependencies:
	gdcm - requires gdcm>=3.0.10
	pylibjpeg - requires pylibjpeg>=2.0 and pylibjpeg-libjpeg>=2.1
Error:Unable to decompress 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])' pixel data because all plugins are missing dependencies:
	gdcm - requires gdcm>=3.0.10
	pylibjpeg - requires pylibjpeg>=2.0 and pylibjpeg-libjpeg>=2.1
Error:Unable to decompress 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])' pixel data because all plugins are missing dependencies:
	gdcm - requires gdcm>=3.0.10
	pylibjpeg - requires pylibjpeg>=2.0 and pylibjpeg-libjpeg>=2.1
Error:Unable to decompress 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])' pixel data because all plugins are missing dependencies:
	gdcm - requires g

Training:  88%|████████▊ | 14/16 [11:46<01:40, 50.44s/it]


ValueError: Caught ValueError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/tmp/ipython-input-1183870778.py", line 64, in __getitem__
    slices = np.stack(slices)  # original
             ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/numpy/_core/shape_base.py", line 444, in stack
    raise ValueError('need at least one array to stack')
ValueError: need at least one array to stack
